In [ ]:
#This code creates indexes on LAR tables to expedite queries



In [4]:
from collections import OrderedDict
import json
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import psycopg2




In [11]:
def connect():
    """Creates a connection to a local PG database."""
    #parameter format for local use
    params = {
    'dbname':'hmda',
    'user':'roellk',
    'password':'',
    'host':'localhost',}
    try:
        conn = psycopg2.connect(**params)
    except psycopg2.Error as e: #if database connection results in an error print the following
        print("I am unable to connect to the database: ", e)
    return conn.cursor(), conn #returns connection and cursor

def create_index(table, column, index_name=""):
    """Creates an index on the specified table and column"""
    sql = """CREATE INDEX {index} ON {table} ({column}); COMMIT;""".format(index=index_name, table=table, column=column)
    cur, conn = connect()
    cur.execute(sql,)
    conn.close()
    
def create_multi_index(table, columns, index_name=""):
    """Creates and index on the specified table and columns. Columns argument is a list"""
    cols = ", ".join(columns)
    sql = """CREATE INDEX {index} ON {table} ({columns}); COMMIT;""".format(index=index_name, table=table, columns=cols)
    cur, conn = connect()
    cur.execute(sql,)
    conn.close()

In [13]:
years = list(range(2004,2017))
years = [str(x) for x in years]
single_index_fields = ["action_type", "app_sex", "co_app_sex", "income", "purchaser_type", "property_type","loan_amount"
                      ,"preapproval", "app_ethnicity", "co_app_ethnicity", "hoepa", "lien_status", "rate_spread"]
for year in years:
    table = "lar_" + year + "_ffiec"
    print(table)
    for idx in single_index_fields:
        create_index(table, idx)
    #create_index(table, "loan_amount")
    create_multi_index(table, ["loan_type", "property_type", "loan_purpose", "action_type", "occupancy", "lien_status"])
    create_multi_index(table, ["loan_type", "property_type", "loan_purpose", "lien_status"])
    create_multi_index(table, ["income", "loan_amount"])
    create_multi_index(table, ["app_race_1", "app_race_2", "app_race_3", "app_race_4", "app_race_5", 
                          "co_app_race_1", "co_app_race_2", "co_app_race_3", "co_app_race_4", "co_app_race_5"])
    create_multi_index(table, ["app_ethnicity", "co_app_ethnicity"])
    

lar_2004_ffiec
lar_2005_ffiec
lar_2006_ffiec


OperationalError: could not extend file "base/16384/491665272": No space left on device
HINT:  Check free disk space.


In [ ]:
create_index("loan_amount", "lar_2004_ffiec", "loan_type")

In [1]:
lar_fields = ["action_type", "msa", "state_code", "county_code", "census_tract", "app_sex", 
       "co_app_sex", "income", "purchaser_type", "denial_1", "denial_2", "denial_3", 
       "edit_status", "property_type", "preapproval", "app_ethnicity", "co_app_ethnicity", 
       "app_race_1", "app_race_2", "app_race_3", "app_race_4", "app_race_5", "co_app_race_1", 
       "co_app_race_2", "co_app_race_3", "co_app_race_4", "co_app_race_5", "rate_spread", 
       "hoepa", "lien_status"]